Make certain you are on a GPU Runtime first, by going to Runtime and selecting "Change Runtime Type", and then choosing Hardware Accelerator as GPU.

In this work, you will build an emotion classifier based on a Huggingface emotions dataset.

You will need to install 🤗 Transformers, numpy and 🤗 Datasets. Run the following three cells.

In [1]:
pip install datasets==1.3.0

     |████████████████████████████████| 184kB 19.0MB/s 
     |████████████████████████████████| 245kB 44.5MB/s 
     |████████████████████████████████| 112kB 50.9MB/s 


In [2]:
pip install transformers==4.3.2

     |████████████████████████████████| 1.8MB 18.6MB/s 
     |████████████████████████████████| 3.2MB 48.2MB/s 
     |████████████████████████████████| 890kB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1a58f2d370293c9486aaff3eb219b679ae8f602b07faec413714939b12e81066
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
pip install numpy==1.20.1

     |████████████████████████████████| 15.3MB 212kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [4]:
from datasets import load_dataset
emotions_dataset = load_dataset('go_emotions', 'simplified')



Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e. Subsequent calls will reuse this data.


To get a sense of what the data looks like, the following function 
will show some examples picked randomly in the dataset.

In [5]:

import datasets
import random
import pandas as pd
from IPython.display import display, HTML



In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))



In [7]:
# For simplicity, train model with single label for item.
restricted_dataset = emotions_dataset#.filter(lambda x: len(x["labels"]) == 1 and 27 not in x['labels'])
print(len(restricted_dataset['train']))
show_random_elements(restricted_dataset['train'])

# For simplicity, train model with single label for item.
restricted_dataset = emotions_dataset.filter(lambda x: len(x["labels"]) == 1 and 27 not in x['labels'])
print(len(restricted_dataset['train']))
show_random_elements(restricted_dataset['train'])


# The emotions are provided as numeric labels. These are the actual orderings, beginning at 0 for admiration:

labels = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", 
          "confusion", "curiosity", "desire", "disappointment", "disapproval",
          "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief",
          "joy", "love", "nervousness", "optimism", "pride", "realization",
          "relief", "remorse", "sadness", "surprise", "neutral"]

          
index_to_labels = {index: label for index, label in enumerate(labels)}

print(index_to_labels)

43410


,id,labels,text
0,efb2f53,[4],When I'm the fifth Hokage I'm gonna make sure no one has to go through that sort of abuse! Believe it!!
1,eepuojs,[0],So fucking awesome. Cheers from Brazil.
2,ed0dtnw,[2],??? Where the hell is this article getting it's data from...
3,eeoju3r,"[3, 20]","I hope your brother is okay. Don’t listen to this guy, he’s a cunt."
4,edxei3w,"[1, 27]",Colts Offense is laughable right now
5,ed5lhu6,[10],I'm just reminding everyone. you're not helping the situation by being pedantic
6,eduzroe,[27],"He missed my personal favorite of ""harridan"" though."
7,edwivg3,[17],And then you got a little vacation from work too! :D
8,ed23i73,[14],[NAME] combat is terrible and the only reason I don’t play it anymore
9,edgaypf,"[0, 18]",I loved how [NAME] feels like everyone in the office is her family. Her relationship with [NAME] is so cute and innocent



23485


,id,labels,text
0,ee8oo9m,[4],To be fair he probably is saying that.
1,ed6ztde,[2],Good. Stay the fuck away.
2,ed1iqnp,[20],"For what it's worth, I hope you have a great year anyway."
3,ed6l9pt,[24],"Sorry guys, I went a little far... You can read my apology post here."
4,ef8aotu,[3],I almost pissed myself waiting so long in the tunnel. Not a fun feeling
5,edrt0xz,[5],do your own homework
6,eevr8sl,[1],I cackled at [NAME] face down in the corner.
7,ed9bsv5,[2],"Shut up, Mag!"
8,ee18p09,[6],Who is afraid of who here? The shower or you?
9,ed1ykey,[7],Enjoying oneself automatically makes an action harmless?


{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [8]:
 from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

# model_name = 'distilbert-base-uncased'
# config = AutoConfig.from_pretrained(model_name, num_labels=27)
# model = AutoModelForSequenceClassification.from_config(config=config)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)



In [9]:
from transformers import AutoConfig, BertTokenizer, BertForSequenceClassification
import torch
model_name = 'bert-base-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=27)
model = AutoModelForSequenceClassification.from_config(config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


In [ ]:
# # from transformers import XLNetTokenizer, XLNetForSequenceClassification
# import torch
# model_name = 'xlnet-base-cased'
# config = AutoConfig.from_pretrained(model_name, num_labels=27)
# model = AutoModelForSequenceClassification.from_config(config=config)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.


In [10]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

def convert_labels_to_int(example):
    example['labels'] = example['labels'][0]
    return example

encoded_dataset = restricted_dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.map(convert_labels_to_int)

encoded_dataset = restricted_dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.map(convert_labels_to_int)



Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-3b02967efc81e313.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-f59190c5f0fce213.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-ef868f095002c837.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-2927be29030058d5.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-24477ddd8dd8a090.arrow
Loading cached processed dataset at

In [11]:
show_random_elements(encoded_dataset['train'])

,attention_mask,id,input_ids,labels,text,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1]",ed10qwp,"[101, 2204, 6735, 2026, 2767, 999, 102]",20,Good luck my friend!,"[0, 0, 0, 0, 0, 0, 0]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eet08tq,"[101, 1998, 1045, 4299, 1045, 2347, 1005, 1056, 15116, 2098, 1999, 2007, 2047, 3933, 1012, 102]",8,And I wish I wasn't lumped in with new jersey.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",ef9b0mm,"[101, 2001, 2023, 2012, 13244, 17479, 1029, 1045, 2359, 2000, 2202, 1037, 27263, 2021, 2018, 2026, 11875, 2006, 102]",6,Was this at Meadowvale? I wanted to take a pic but had my gloves on,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",efc9012,"[101, 2064, 12210, 2004, 2026, 2034, 2051, 4012, 28120, 2075, 2011, 9055, 1999, 1996, 4542, 2045, 2001, 16889, 1010, 2059, 7167, 1997, 8505, 19947, 1998, 7407, 1010, 2179, 2009, 2200, 2200, 17115, 1012, 102]",14,"Can confirm as my first time commuting by motorcycle in the rain there was hail, then lots of thunder bolts and lightning , found it very very frightening.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eefiwf7,"[101, 2092, 1045, 1005, 1049, 6015, 2004, 4485, 2292, 1005, 1055, 3246, 2005, 1996, 2190, 1010, 17770, 2323, 2022, 2067, 1006, 1029, 1007, 102]",14,"Well I'm scared as shit let's hope for the best, benz should be back(?)","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eddmdvh,"[101, 2347, 1005, 1056, 3480, 3993, 2012, 2035, 1012, 2009, 2371, 16334, 2000, 2033, 3752, 2009, 1012, 2057, 1005, 2128, 2035, 2204, 1012, 102]",4,Wasn't hurtful at all. It felt comforting to me reading it. We're all good.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",edkc35l,"[101, 1045, 2001, 2036, 7087, 6340, 2012, 2147, 1012, 1045, 2001, 2006, 22157, 2005, 1037, 2096, 1012, 2017, 1005, 2128, 2025, 2894, 1012, 102]",25,I was also bullied at work. I was on lithium for a while. You're not alone.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eep759o,"[101, 25212, 4095, 1010, 2428, 3374, 2005, 2115, 3279, 1012, 102]",25,"Eesh, really sorry for your loss.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eet0t1s,"[101, 7929, 1012, 1012, 1012, 1012, 2002, 2064, 1008, 2655, 1008, 2009, 1996, 2110, 1997, 1996, 2586, 1010, 2021, 2008, 2987, 1005, 1056, 2812, 2009, 2003, 1012, 102]",10,"Ok.... he can *call* it the State of the Union, but that doesn't mean it is.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",eesvfvp,"[101, 2009, 1521, 1055, 16325, 2000, 6011, 2008, 2242, 6526, 2084, 2000, 6011, 2008, 2242, 2038, 2467, 5839, 1010, 1051, 16665, 2213, 2003, 2006, 2026, 2217, 8840, 2140, 102]",1,"It’s simpler to prove that something exists than to prove that something has always existed, Occam is on my side lol","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:


In [12]:
## Fine-tuning the model
from transformers import Trainer, TrainingArguments
metric_name = "accuracy"

# args = TrainingArguments(
#       "test-emotions",
#       evaluation_strategy = "epoch",
#       learning_rate=0.00001,
#       per_device_train_batch_size=32,
#       per_device_eval_batch_size=32,
#       num_train_epochs=20,
#       weight_decay=0.01,
#       load_best_model_at_end=True,
#       metric_for_best_model=metric_name,
#   )
args = TrainingArguments(
    "test-emotions",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.


In [13]:

metric = datasets.load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Then we just need to pass all of this along with our datasets to the `Trainer`:

validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [14]:
import numpy as np
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,2.650200,2.054448,0.436401,6.894300,428.763000
2,2.109600,1.819487,0.504060,6.897400,428.569000


TrainOutput(global_step=1468, training_loss=2.2262518776210194, metrics={'train_runtime': 376.393, 'train_samples_per_second': 3.9, 'total_flos': 1107159458636304, 'epoch': 2.0})

In [15]:


# We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one). We also run a sample prediction to demonstrate the API:

trainer.evaluate()

prepared_input = tokenizer.prepare_seq2seq_batch(["I am very sad"], return_tensors='pt')
model = model.to('cpu')
model.eval()
model_output = model(**prepared_input)
prediction = np.argmax(model_output.logits[0].detach().numpy())
index_to_labels[prediction]



'sadness'